In [1]:
import numpy as np

In [2]:
from lingualytics.preprocessing import remove_lessthan, remove_punctuation, remove_stopwords
from lingualytics.stopwords import hi_stopwords,en_stopwords
from texthero.preprocessing import remove_digits
import pandas as pd

In [3]:
DATASET = r"D:\DOCUMENTS\faq-virtual-agent-main\faqs.csv"

In [4]:
df = pd.read_csv(DATASET)
df


,Q,A
0,What is kandi?,kandi (pronounced kandee) is a platform that h...
1,Have feedback or want to know more?,We are a passionate set of application focused...
2,What components does kandi cover?,kandi helps you select software components acr...
3,How do I use kandi?,kandi provides two simplified experiences to h...
4,How do I shortlist components on kandi?,You can use the below filters to shortlist com...
5,How do I implement the components that I have ...,The component listing and detailed insights pa...


In [5]:
# pd.set_option('display.max_colwidth', None)
df['clean_Q'] = df['Q'].pipe(remove_digits) \
                                    .pipe(remove_punctuation) \
#                                    .pipe(remove_lessthan,length=3) \
#                                    .pipe(remove_stopwords,stopwords=en_stopwords.union(hi_stopwords))
df

C:\Users\jaswa\anaconda3\lib\site-packages\lingualytics\preprocessing.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(rf"([{punctuation}])+", " ")


,Q,A,clean_Q
0,What is kandi?,kandi (pronounced kandee) is a platform that h...,What is kandi
1,Have feedback or want to know more?,We are a passionate set of application focused...,Have feedback or want to know more
2,What components does kandi cover?,kandi helps you select software components acr...,What components does kandi cover
3,How do I use kandi?,kandi provides two simplified experiences to h...,How do I use kandi
4,How do I shortlist components on kandi?,You can use the below filters to shortlist com...,How do I shortlist components on kandi
5,How do I implement the components that I have ...,The component listing and detailed insights pa...,How do I implement the components that I have ...


In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [7]:
q_embs = model.encode(df['clean_Q'], convert_to_tensor=True)

C:\Users\jaswa\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [8]:
q_embs.shape

torch.Size([6, 384])

In [9]:
usr_query = "tell me about kandi?"

In [10]:
df_usr_query = pd.DataFrame([usr_query], columns=["usr_query"])
df_usr_query

,usr_query
0,tell me about kandi?


In [11]:
df_usr_query['clean_usr_Q'] = df_usr_query['usr_query'].pipe(remove_digits) \
                                    .pipe(remove_punctuation)
df_usr_query

C:\Users\jaswa\anaconda3\lib\site-packages\lingualytics\preprocessing.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(rf"([{punctuation}])+", " ")


,usr_query,clean_usr_Q
0,tell me about kandi?,tell me about kandi


In [12]:
usr_q_emb = model.encode(df_usr_query['clean_usr_Q'], convert_to_tensor=True)
usr_q_emb.shape

C:\Users\jaswa\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


torch.Size([1, 384])

In [13]:
usr_q_emb

tensor([[ 0.2014,  0.3499, -0.4456, -0.5973, -0.3234, -0.2091,  0.6431,  0.4012,
         -0.1202,  0.4589,  0.5733, -0.8320, -0.0467, -0.4634,  0.1831, -0.8133,
          0.1204, -0.1574, -0.2324, -0.3424,  0.1234,  0.7537,  0.4026, -0.1467,
          0.0680, -0.0586,  0.7823,  0.1323, -0.0167, -0.5083, -0.0806,  0.2821,
         -0.0444, -0.1481,  0.3830,  0.5527, -0.2032,  0.0954, -0.3822, -0.4040,
          0.4833, -0.4490,  0.0834, -0.0017,  0.2304,  0.0032, -0.4206, -0.4436,
         -0.3475,  0.2036, -0.0677, -0.1146,  0.9707,  0.0749,  0.4746, -0.5553,
          0.1421,  0.9708,  0.2575, -0.3171,  0.1675, -0.0162, -0.2684,  0.5456,
          0.2091, -0.7014,  0.1628,  0.1007, -0.1575, -0.8526,  0.0253, -0.2285,
         -0.0116,  0.0445, -0.4219, -0.1486, -0.3311, -0.2833, -0.2604, -0.6911,
          0.0506,  0.6993, -0.2380, -0.4244, -0.1133,  0.3774,  0.2479, -0.3001,
          0.0999, -0.2378,  0.3417,  0.3603, -0.0207,  0.4266, -0.2073, -0.2826,
          0.0976, -0.2696, -

In [14]:
import torch

In [15]:
cos_fn = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

In [16]:
idx = np.argmax(cos_fn(usr_q_emb, q_embs)).item()

In [17]:
df['A'][idx]

'kandi (pronounced kandee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items.'

In [18]:
def pred_answer(usr_query):
    df_usr_query = pd.DataFrame([usr_query], columns=["usr_query"])
    df_usr_query['clean_usr_Q'] = df_usr_query['usr_query'].pipe(remove_digits) \
                                    .pipe(remove_punctuation)
    usr_q_emb = model.encode(df_usr_query['clean_usr_Q'], convert_to_tensor=True)
    cos_fn = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    idx = np.argmax(cos_fn(usr_q_emb, q_embs)).item()
    return df['A'][idx]
    

In [ ]:
while True:
    input("Ask a query: ")
    if usr_query == "exit":
        break
    else:
        print(pred_answer(usr_query))
        print("------------")